# Crawling Data Twitter dan Web

## Hasil 

[link drive hasil crawling](https://drive.google.com/drive/folders/1-5qgX_yas_K-pkFDEuWH9uCgur8ydpnQ?usp=sharing)

## Twitter

### Snscrape

In [ ]:
_= !pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
def tweet_scrape(tag, count=100):
  list_tweet = []
  for i,tweet in enumerate(sntwitter.TwitterHashtagScraper(tag).get_items()):
    if i > count:
      break
    list_tweet.append({
      'Nama': tweet.user.username, 
      'Content': tweet.content, 
      'Lokasi': tweet.user.location
    })
  return pd.DataFrame(list_tweet) 

In [ ]:
tweet = tweet_scrape('capres2024 AND puanmaharani')
tweet.to_csv('puanScrape.csv', index=False)

<ipython-input-3-cbc57a5e4014>:8: FutureWarning: content is deprecated, use rawContent instead
  'Content': tweet.content,


### Twint

In [ ]:
# _= !pip3 uninstall twint -y && pip3 install twint && pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
# _= !pip install nest_asyncio

In [ ]:
# import twint
# import nest_asyncio
# nest_asyncio.apply()

In [ ]:
# c = twint.Config()
# c.Search = '#capres2024 AND #puanmaharani'
# c.Lang = 'id'
# c.Limit = 1000
# # c.Since = "2019-01-01"
# # c.Until = "2022-01-01"
# c.Pandas = True
# twint.run.Search(c)

In [ ]:
# Tweets_df = twint.storage.panda.Tweets_df

In [ ]:
# Tweets_df.columns

In [ ]:
# df = pd.DataFrame({
#     'name': Tweets_df['name'],
#     'tweet': Tweets_df['tweet'],
#     'place': Tweets_df['place'],
# })

In [ ]:
# df.to_csv('ganjarCapres.csv')

## Web

In [17]:
_= !pip install bs4

In [18]:
import requests

In [19]:
from bs4 import BeautifulSoup

In [20]:
import pandas as pd

### pta.trunojoyo.ac.id

In [26]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-1].a['href'].split('/')[-1])

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        link = artikel.find('a', class_='gray button')['href']
        respone = requests.get(link)
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nim = link[-12:]
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = ' '.join(cont.find('a', class_='title').get_text().splitlines()).capitalize()
        abstrak = ' '.join(cont.find('p').get_text().splitlines()).capitalize()
        if abstrak == '':
          abstrak = ' '.join(cont.find('p').findNext('p').stripped_strings).capitalize()
        # judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        # abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'NPM' : nim,
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [27]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [28]:
scPta.get_berita()

In [29]:
scPta.result()

,NPM,Penulis,Judul,Abstrak
0,040411100468,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database ...,Sistem informasi akademik (siakad) merupaka...
1,040411100476,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,040411100480,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untuk enk...,Web server adalah sebuah perangkat lunak serve...
3,070411100070,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,080411100115,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...
853,160411100032,Rachmad Agung Pambudi,Penerapan algoritma long-short term memory unt...,Investasi saham selama ini memiliki resiko ker...
854,160411100182,Nadila Hidayanti,Sistem pencarian teks al-quran terjemahan berb...,Information retrieval (ir) merupakan pengambil...
855,160411100077,Afni Sakinah,Klasifikasi kompleksitas visual citra sampah m...,Klasifikasi citra merupakan proses pengelompok...
856,160411100084,Friska Fatmawatiningrum,Identifikasi biner atribut pejalan kaki menggu...,Identifikasi atribut pejalan kaki merupakan sa...


In [30]:
scPta.save()

Berhasil di save


### Kompas

In [ ]:
class ScrapingKompas:
  def __init__(self, url):
    self.url = url

  def page(self):
    res_ = requests.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='paging__link')[:-2]
    return len(contents_)

  def artikel_page(self, page):
    res_ = requests.get(f'{self.url}?page={page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='article__link')
    return contents_

  def get_berita(self):
    berita_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        res = requests.get(artikel['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # ambil data
        judul = soup.find('h1', class_='read__title').string
        isi = isi = ' '.join(soup.find('div', class_='read__content').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list
    return 
  
  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaKompas'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
data = ScrapingKompas('https://indeks.kompas.com')

In [ ]:
data.get_berita()

In [ ]:
data.result()

,Judul,Isi Berita
0,"All England 2023: Hasil Drawing, Jadwal, dan T...",KOMPAS.com - Indonesia mengirim 14 wakilnya un...
1,Jokowi Klaim Tidak Ada Kendala Pembebasan Laha...,"BALIKPAPAN , KOMPAS.com - Presiden Joko Widodo..."
2,Camat Penjaringan Larang Ketua RT di Pluit Bon...,"JAKARTA, KOMPAS.com - Kepala Seksi (Kasi) Peme..."
3,Jenis Bunga yang Bisa Mengusir Hama di Sawah,"JAKARTA, KOMPAS.com - Kehadiran hama di sawah ..."
4,"Oknum Polisi di Sulsel ""Bekingi"" Pengedar Sabu...","MAKASSAR, KOMPAS.com - Oknum polisi berinisial..."
5,Mayat Perempuan Ditemukan di Gudang Terbengkal...,"LAMPUNG , KOMPAS.com - Jasad perempuan ditemuk..."
6,"Kemacetan Jakarta “Naik Peringkat”, Pejabat DK...","JAKARTA, KOMPAS.com - Lembaga pemeringkat lalu..."
7,"Gempa M 5,9 Guncang Maluku Barat Daya, Tidak B...","AMBON, KOMPAS.com- Gempa tektonik berkekuatan ..."
8,Caleg Sistem Proporsional Tertutup Rentan Dit...,"JAKARTA, KOMPAS.com - Pakar hukum tata negara ..."
9,"Beras Masih Mahal, Gus Muhaimin Minta Pemerint...",KOMPAS.com - Wakil Ketua Dewan Perwakilan Raky...


In [ ]:
data.save()

Berhasil di save


### Detik.com

In [ ]:
class ScrapingDetik:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res_ = requests.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.find('div', class_='pagination').findChildren()
    return int(contents_[-2].text)

  def artikel_page(self, page):
    res_ = requests.get(f'{self.url}/{page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='media__link')
    return contents_
  
  def get_berita(self):
    berita_list = []
    page = 5  # self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in range(1, len(artikels), 2):
        res = requests.get(artikels[artikel]['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # hapus tag style script
        for d in soup(['style', 'script']):
          d.decompose()
        # ambil data
        judul = (soup.find('h1', class_='detail__title'))
        isi = (soup.find('div', class_='detail__body-text'))
        if judul == None and isi == None:
          continue
        berita_list.append({
            'Judul': ' '.join(judul.stripped_strings),
            'Isi Berita': ' '.join(isi.stripped_strings)
        })
    self.beritas = berita_list

  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaDetik'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
scrap = ScrapingDetik('https://news.detik.com/indeks')

<a class="media__link" href="https://news.detik.com/pemilu/d-6575185/ketum-ppp-ngaku-siap-hadapi-pemilu-proporsional-terbuka-atau-tertutup" onclick='_pt(this, "newsfeed", "Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup", "artikel 1")'>
<span class="ratiobox ratiobox--4-3 lqd">
<img alt="Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup" class="" src="https://awsimages.detik.net.id/community/media/visual/2023/02/17/ketum-ppp-mardiono_43.jpeg?w=210&amp;q=90" title="Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup"> </img></span>
</a>,

In [ ]:
scrap.get_berita()

In [ ]:
scrap.result()

,Judul,Isi Berita
0,Heru Budi soal Jokowi Mau Normalisasi Ciliwung...,Jakarta - Presiden Joko Widodo (Jokowi) ingin ...
1,Sidang Etik Ricky Rizal Digelar Usai Keputusan...,Jakarta - Majelis sidang Komisi Kode Etik Polr...
2,3 Halte TransJ Ditutup Imbas Proyek MRT Fase 2...,Jakarta - PT Transportasi Jakarta ( TransJakar...
3,Daftar Putusan Etik di Pusaran Kasus Yosua: Sa...,Jakarta - Komisi Kode Etik Polri (KKEP) terus ...
4,Kajati Jabar Asep Mulyana Dilantik Jadi Dirjen...,Jakarta - Kepala Kejaksaan Tinggi (Kajati) Jaw...
...,...,...
94,Ketua DPR Rusia Tuding AS Sudah Hancurkan Stab...,Moskow - Ketua Duma atau majelis rendah parlem...
95,"Mundur sebagai Menpora, Zainudin Amali Ngaku S...",Jakarta - Zainudin Amali mengaku sudah memberi...
96,Anak Pejabat Pajak Tersangka Penganiayaan Tera...,Jakarta - Polisi menetapkan Mario Dandy Satrio...
97,Kejari Denpasar Musnahkan Barang Bukti Narkoti...,Denpasar - Kejaksaan Negeri Denpasar melakukan...


In [ ]:
scrap.save()

Berhasil di save
